In [69]:
# 모듈 로딩
# - Model 실행 관련
import torch              
import torch.nn as nn              
import torch.nn.functional as F 
# from torchinfo import summary 
import os
import pickle

In [70]:
MODEL_FILE = r'C:\Users\KDP-23\Desktop\KDT6\NLP\project\cgi-bin\best_model4.pth'
VOCAB_FILE = r'C:\Users\KDP-23\Desktop\KDT6\NLP\project\models\vocab.txt'
max_length = 50

In [71]:
def load_model(MODEL_FILE):
   if os.path.exists(MODEL_FILE):
      model = torch.load(MODEL_FILE)
      return model

   else:
      result = '파일이 없습니다.'
      return result

In [72]:
import re
stop_words = 'basic_ko_stopwords.txt'

# 한글만 토큰화
def getToken(textlist,tokenizer):
    text_to_token=[]
    for idx,text in enumerate(textlist):
        # 한글빼고 다지우기
        text=re.sub('[^ㄱ-ㅎ가-힣]+',' ',text)

        # 토큰 추출
        tokens=tokenizer.morphs(text,norm=False, stem=False)
        for token in tokens:
            # stop word 체크
             if token in stop_words:
                  tokens.remove(token)
        text_to_token.append(tokens)

    return text_to_token

In [115]:
stop_words

'basic_ko_stopwords.txt'

In [73]:
import numpy as np

## 패딩
def pad_sequences(sequences, max_length, pad_value):
    result = list()
    for sequence in sequences:
        sequence = sequence[:max_length]
        pad_length = max_length - len(sequence)
        padded_sequence = sequence +[pad_value] * pad_length
        result.append(padded_sequence)
    return np.asarray(result)

In [87]:
ex_text ="진짜 너무하시네요 배송이늦는건 엄절수없다하고 참앗지만 밑에 서랍이 금방 안되서 AS되냐고 물어봣는데 전화도 안받고 10시에 톡을보냇는데 오후6시에 답해서 뭐하나물어보려고하니 상담시간이아니라고 뜨네요 빨리as좀 해주세요 안되면 안된다고 답장부탁드립니"

In [88]:
from konlpy.tag import *
tokenizer = Okt()
text_tokens = [ token for token in tokenizer.morphs(ex_text, norm=False, stem=False)]
print(text_tokens)

['진짜', '너무하시네요', '배송', '이', '늦는건', '엄', '절수', '없다하고', '참앗지만', '밑', '에', '서랍', '이', '금방', '안되서', 'AS', '되냐고', '물어', '봣', '는데', '전화', '도', '안', '받고', '10시', '에', '톡', '을', '보냇', '는데', '오후', '6시', '에', '답', '해서', '뭐', '하', '나', '물어보려고하니', '상담시', '간', '이', '아니라고', '뜨네요', '빨리', 'as', '좀', '해주세요', '안되면', '안된다고', '답장', '부탁드립니']


In [89]:
with open(VOCAB_FILE, 'r', encoding='utf-8') as f:
    vocab = f.readlines()
    id_to_token ={idx: token.replace('\n','') for idx, token in enumerate(vocab)}
    token_to_id ={token.replace('\n',''):  idx for idx, token in enumerate(vocab)}
    print(id_to_token)

{0: '<pad>', 1: '<unk>', 2: '이', 3: '도', 4: '에', 5: '가', 6: '잘', 7: '너무', 8: '배송', 9: '은', 10: '요', 11: '좋아요', 12: '안', 13: '는', 14: '재구매', 15: '구매', 16: '다', 17: '사용', 18: '을', 19: '으로', 20: '그냥', 21: '로', 22: '한', 23: '제품', 24: '했는데', 25: '가격', 26: '보다', 27: '생각', 28: '것', 29: '하고', 30: '같아요', 31: '입니다', 32: '좀', 33: '주문', 34: '맛', 35: '고', 36: '많이', 37: '더', 38: '못', 39: '를', 40: '해서', 41: '별로', 42: '때', 43: '만', 44: '사이즈', 45: '들', 46: '개', 47: '포장', 48: '에서', 49: '서', 50: '합니다', 51: '네', 52: '거', 53: '상품', 54: '좋네요', 55: '했어요', 56: '빠르고', 57: '정말', 58: '인데', 59: '냄새', 60: '다른', 61: '부분', 62: '제', 63: '이라', 64: '나', 65: '좋습니다', 66: '하나', 67: '넘', 68: 'ㅎㅎ', 69: '아주', 70: '두', 71: '진짜', 72: '샀는데', 73: '저', 74: '만족합니다', 75: '할', 76: '좋은', 77: '조금', 78: '구입', 79: '또', 80: '반품', 81: '아이', 82: '정도', 83: '딱', 84: '해', 85: '느낌', 86: '아직', 87: '맘', 88: '감사합니다', 89: '중', 90: '완전', 91: '엄청', 92: '좋고', 93: '과', 94: '처음', 95: '게', 96: '일', 97: '의', 98: '비', 99: '면', 100: '않고', 101: '디자인', 102: 

In [90]:
# from konlpy.tag import *
# tokenizer = Okt()
# text_tokens = getToken(ex_text,tokenizer)

# text_tokens

In [91]:
max_length = 50

In [92]:
key_unk = [key for key, value in id_to_token.items() if value == '<unk>']
unk_id = key_unk[0]
unk_id

1

In [93]:
text_ids = [
    [token_to_id.get(token, unk_id) for token in review] for review in text_tokens]

In [94]:
key_pad = [key for key, value in id_to_token.items() if value == '<pad>']
pad_id = key_pad[0]
pad_id


0

In [95]:
text_ids = pad_sequences(text_ids, max_length, pad_id)

In [96]:
text_ids = torch.tensor(text_ids)

In [97]:
import torch.nn as nn

class SentenceClassifier(nn.Module):
    def __init__(self,
                 n_vocab,
                 hidden_dim,
                 embedding_dim,
                 n_layers, 
                 dropout = 0.5,
                 bidirectional = True,
                 model_type = 'lstm'):
        super().__init__()

        self.embedding = nn.Embedding(
            num_embeddings=n_vocab,
            embedding_dim = embedding_dim,
            padding_idx= 0)
        
        if model_type =='rnn':
            self.model = nn.RNN(
                input_size=embedding_dim,
                hidden_size = hidden_dim,
                num_layers= n_layers,
                bidirectional=bidirectional,
                dropout=dropout,
                batch_first=True)
    
        elif model_type == "lstm":
            self.model = nn.LSTM(
                input_size=embedding_dim,
                hidden_size = hidden_dim,
                num_layers= n_layers,
                bidirectional=bidirectional,
                dropout=dropout,
                batch_first=True )
            
        if bidirectional:
            self.classifier = nn.Linear(hidden_dim * 2, 1)
            
        else:
            self.classifier = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(dropout)

    def forward(self, inputs):
        embeddings =self.embedding(inputs)
        output, _ = self.model(embeddings)
        last_output = output[:, -1, :]
        last_output = self. dropout(last_output)
        logits = self. classifier(last_output)
        return logits

In [98]:
mymodel = load_model(MODEL_FILE)

In [99]:
# 예측
mymodel.eval()

with torch.no_grad():
    
    # 예측
    pre_y = mymodel(text_ids)
    yhat = torch.sigmoid(pre_y)>.5

    # 결과 출력 (임계값을 0.5로 설정해 긍정/부정 분류)
    if yhat.all():  
        result = "긍정"
    else:
        result = "부정"
    
print(result)



부정


In [114]:
F= 0
T= 0

for y in yhat:
    if y.item():
        T += 1
    else:
        F += 1

print(F,T)

46 6
